04/22 to 
ML 
1. tackle the problem of units
2. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randn 
import pandas as pd
import seaborn as sns
import matplotlib.ticker as mtick
import time
from scipy.optimize import fmin_l_bfgs_b
from scipy.optimize import minimize
import pandas as pd
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import time
np.random.seed(28)

#plotting setting
size=10;f=12;

from ML_estimator import *
from Evaluation import *
from Basic import *

# Basic functions

# EXP3

In [8]:
#main 
def experiment3(N,lamb,d,targets,noise_std,ml_resol,flag_plot):
    # Array
    arr_dim=d*(N-1) # dimension of the array
    arr=array(N,arr_dim,0) # array elements location
    R_near=near_field_Boundary(lamb,arr_dim)
    arr_center=0.5*arr_dim
    time_ML=0
    #Prepare arrays for results
    num_sam=5
    true_targets=np.empty([len(targets),num_sam,2])
    ML_targets=np.empty([len(targets),num_sam,2])


    #begin experiment
    for i in range (len(targets)):
        if(i%10==0):
            print(i)
        
        for j in range(num_sam):
            true_targets[i,j]=targets[i]
            # phase measurements
            phase_obs=np.zeros(N)
            # phi_0=np.random.uniform(low=0,high=2*np.pi)#phase shift (offset, unknown constant)
            phi_0=1
#             print(f'phase shift is {phi_0}rad')

            dist=np.array([np.linalg.norm(arr[n]-true_targets[i,j]) for n in range(N)])#distance between the elements and target
            phase_obs=(4*np.pi/lamb*dist+phi_0+randn(N)*noise_std)%(2*np.pi)
            phase_shift=(4*np.pi/lamb*dist+phi_0)%(2*np.pi)
            phase_true=(4*np.pi/lamb*dist)%(2*np.pi)
            phase_true_unwrap=(4*np.pi/lamb*dist)

            if(flag_plot==1):
            #plot the phases
                plt.figure(figsize=(4,4))
                plt.scatter(range(1,N+1),phase_obs,color='red',s=size)
                plt.scatter(range(1,N+1),phase_shift,color='purple',s=size)
                plt.scatter(range(1,N+1),phase_true,color='green',s=size)
                plt.scatter(range(1,N+1),phase_true_unwrap,color='blue',s=size)
                plt.legend(['obs','shift','true','true unwrap'])
                plt.title('Target and array')
                plt.xlabel('x axis [Anetenna]')
                plt.ylabel('y axis [rad]')

            xrange=np.array([arr_center-R_near,arr_center+R_near])
            yrange=np.array([0,R_near])
            
            #ML
            T1=time.perf_counter()
            ml=ML();
            _,Loc_MLgrid=ml.ML_grid(xrange,yrange,phase_obs,arr,lamb,resolution=ml_resol,flag_plot=False)
            T2=time.perf_counter()
            time_ML+=(T2-T1)
        #     ml.plotting(size,f,targets,Loc_MLgrid,arr,xrange,yrange)
            ML_targets[i,j]=Loc_MLgrid.reshape(2)
            
            
    print(f'ML processing time per iteration:{time_ML/(len(targets)*num_sam)}')
    return true_targets,ML_targets


In [9]:
#main 
N=10 # number of elements
lamb=2e-3 # wavelength of the signal
d=0.25*lamb # distance between 2 adjacent elements


arr_dim=d*(N-1) # dimension of the array
arr=array(N,arr_dim,0) # array elements location
R_near=near_field_Boundary(lamb,arr_dim)
arr_center=0.5*arr_dim

# generating targets
N_target=100
# targets = sample_in_half_circle(R_near,N_target)+np.array([arr_center,0])
# # print(f"True target:{targets}")
# np.savetxt('targets.txt',targets)
targets=np.loadtxt('targets.txt')
# print(targets)

In [10]:
ML_MSE=np.empty([3,3])
# ML_std=np.empty([3,3])

noise_std=[0.001,0.01,0.1]
ml_resol=[0.5*lamb,0.25*lamb,0.125*lamb]
flag_plot=0;
for i in range(len(noise_std)):
    for j in range(len(ml_resol)):
        true_targets,ML_targets=experiment3(N,lamb,d,targets,noise_std[i],ml_resol[j],flag_plot)
        ML_MSE[i,j],_=evaluate(true_targets,ML_targets)
        print(f'noise_std:{noise_std[i]},ML1 resolution:{ml_resol[j]/lamb}wavelength')
        print(f'ML_mean_Squared_loss: {ML_MSE[i,j]:.4}')



0
10
20
30
40
50
60
70
80
90
ML processing time per iteration:0.02320939000014914
noise_std:0.001,ML1 resolution:0.5wavelength
ML_mean_Squared_loss: -48.17
0
10
20
30


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,0],ML_MSE[1,0],ML_MSE[2,0]]),c='r')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,1],ML_MSE[1,1],ML_MSE[2,1]]),c='b')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,2],ML_MSE[1,2],ML_MSE[2,2]]),c='g')

plt.legend(['ML grid_size=0.5wavelength','ML grid_size=0.25wavelength','ML grid_size=0.125wavelength'])
# plt.ylim([-56,-42])

plt.xlabel('σ-2 [dB]')
plt.ylabel('MSE [dB]')
plt.grid()
# plt.savefig('C:/Users/73160/OneDrive/项目/project/Weekly reports/Sp_5/Exp3_MSE_Result.png')
# np.savetxt('Exp3_result_ML_MSE.txt',ML_MSE,fmt='%.03f')

# EXP 4 synthetic priors

In [ ]:
a=np.zeros([3,2])
print(a.shape[0])

In [ ]:
A=np.zeros((3,2,2))

In [ ]:
import random
print(random.gauss(0,1))

In [ ]:
import random
def Synthetic_prior(target,num_sample,prior_sigma):
    K=len(target)
    prior_means=np.zeros((K,num_sample,2))
    for i in range(K):
        for j in range(num_sample):
            prior_means[i,j,0]=random.gauss(target[i,0],prior_sigma)
            prior_means[i,j,1]=random.gauss(target[i,1],prior_sigma)
    return prior_means


def experiment4(N,lamb,d,targets,noise_std,ml_resol,prior_means,prior_sigma,flag_plot):
    # Array
    arr_dim=d*(N-1) # dimension of the array
    arr=array(N,arr_dim,0) # array elements location
    R_near=near_field_Boundary(lamb,arr_dim)
    arr_center=0.5*arr_dim
    time_ML=0
    #Prepare arrays for results
    num_sam=5
    true_targets=np.empty([len(targets),num_sam,2])
    ML_targets=np.empty([len(targets),num_sam,2])
    ML_targets_prior=np.empty([len(targets),num_sam,2])

    #begin experiment
    for i in range (len(targets)):
        if(i%10==0):
            print(i)
        
        for j in range(num_sam):
            true_targets[i,j]=targets[i]
            # phase measurements
            phase_obs=np.zeros(N)
            # phi_0=np.random.uniform(low=0,high=2*np.pi)#phase shift (offset, unknown constant)
            phi_0=1
#             print(f'phase shift is {phi_0}rad')

            dist=np.array([np.linalg.norm(arr[n]-true_targets[i,j]) for n in range(N)])#distance between the elements and target
            phase_obs=(4*np.pi/lamb*dist+phi_0+randn(N)*noise_std)%(2*np.pi)
            phase_shift=(4*np.pi/lamb*dist+phi_0)%(2*np.pi)
            phase_true=(4*np.pi/lamb*dist)%(2*np.pi)
            phase_true_unwrap=(4*np.pi/lamb*dist)

            if(flag_plot==1):
            #plot the phases
                plt.figure(figsize=(4,4))
                plt.scatter(range(1,N+1),phase_obs,color='red',s=size)
                plt.scatter(range(1,N+1),phase_shift,color='purple',s=size)
                plt.scatter(range(1,N+1),phase_true,color='green',s=size)
                plt.scatter(range(1,N+1),phase_true_unwrap,color='blue',s=size)
                plt.legend(['obs','shift','true','true unwrap'])
                plt.title('Target and array')
                plt.xlabel('x axis [Anetenna]')
                plt.ylabel('y axis [rad]')
            
            #search space
            xrange=np.array([arr_center-R_near,arr_center+R_near])
            yrange=np.array([0,R_near])
            
            prior_mean=prior_means[i,j]
            xrange_p=np.array([prior_mean[0]-3*prior_sigma,prior_mean[0]+3*prior_sigma])
            yrange_p=np.array([prior_mean[1]-3*prior_sigma,prior_mean[1]+3*prior_sigma])
#             xrange_p[0]=max(xrange[0],xrange_p[0]);xrange_p[1]=min(xrange[1],xrange_p[1]);
#             yrange_p[0]=max(yrange[0],yrange_p[0]);yrange_p[1]=min(yrange[1],yrange_p[1]);
            
            #ML
#             T1=time.perf_counter()
            ml=ML();
            num_grid,Loc_MLgrid=ml.ML_grid(xrange,yrange,phase_obs,arr,lamb,resolution=ml_resol,flag_plot=False)
            num=num_grid[0]*num_grid[1]
#             T2=time.perf_counter()
#             time_ML+=(T2-T1)
        #     ml.plotting(size,f,targets,Loc_MLgrid,xrange,yrange)
            ML_targets[i,j]=Loc_MLgrid.reshape(2)        
            
            
            #ML prior bound
#             T1=time.perf_counter()
            ml=ML();
            num_grid,Loc_MLgrid_prior=ml.ML_grid(xrange_p,yrange_p,phase_obs,arr,lamb,resolution=ml_resol,flag_plot=False)
            num_prior=num_grid[0]*num_grid[1]
#             T2=time.perf_counter()
#             time_ML+=(T2-T1)
        #     ml.plotting(size,f,targets,Loc_MLgrid,xrange,yrange)
            ML_targets_prior[i,j]=Loc_MLgrid_prior.reshape(2)  
            
            
#     print(f'ML processing time per iteration:{time_ML/(len(targets)*num_sam)}')
    return true_targets,ML_targets,ML_targets_prior,num,num_prior


In [ ]:
#main 
N=10 # number of elements
lamb=2e-3 # wavelength of the signal
d=0.25*lamb # distance between 2 adjacent elements


arr_dim=d*(N-1) # dimension of the array
arr=array(N,arr_dim,0) # array elements location
R_near=near_field_Boundary(lamb,arr_dim)
arr_center=0.5*arr_dim

# generating targets
N_target=100
# targets = sample_in_half_circle(R_near,N_target)+np.array([arr_center,0])
# # print(f"True target:{targets}")
# np.savetxt('targets.txt',targets)
targets=np.loadtxt('targets.txt')
# print(targets)


In [ ]:
#generate synthetic priors
prior_sigma=1e-3
prior_means=Synthetic_prior(targets,5,prior_sigma)
print(prior_means.shape)

#illustrations

ML_MSE=np.empty([3,3])
# ML_std=np.empty([3,3])

noise_std=0.001
ml_resol=0.5*lamb
flag_plot=0;



fig=plt.figure()
ax=fig.add_subplot(1,1,1)


true_targets,ML_targets,ML_targets_prior,num,num_prior=experiment4(N,lamb,d,targets[0:2],noise_std,ml_resol,prior_means[0:2],prior_sigma,flag_plot)
ML_MSE[0,0],_=evaluate(true_targets,ML_targets_prior)
print(f'noise_std:{noise_std},ML1 resolution:{ml_resol/lamb}wavelength')
print(f'ML_mean_Squared_loss: {ML_MSE[0,0]:.4}')
print(num,num_prior)




near_field_square=plt.Rectangle(xy=(arr_center-R_near,0),height=R_near,width=2*R_near,color='gray')
square_1=plt.Rectangle(xy=(prior_means[0,0,0]-3*prior_sigma,prior_means[0,0,1]-3*prior_sigma),height=prior_sigma*6,width=6*prior_sigma,color='yellow')

ax.add_patch(near_field_square)
ax.add_patch(square_1)
plt.scatter(targets[0,0],targets[0,1],c='green')
plt.scatter(prior_means[0,0,0],prior_means[0,0,1],c='r')
plt.scatter(ML_targets[0,0,0],ML_targets[0,0,1],c='black')
plt.scatter(ML_targets_prior[0,0,0],ML_targets_prior[0,0,1],c='blue')
plt.axis('equal')
plt.legend(['near-field','new search space based on prior','true target location','prior_mean','estimated location','estimated location prior'])



In [ ]:
#generate synthetic priors
prior_sigma=1e-3
prior_means=Synthetic_prior(targets,5,prior_sigma)
print(prior_means.shape)

#EXP4
ML_MSE=np.empty([3,3,2])
noise_std=[0.001,0.01,0.1]
ml_resol=[0.5*lamb,0.25*lamb,0.125*lamb]
flag_plot=0;
for i in range(len(noise_std)):
    for j in range(len(ml_resol)):
        true_targets,ML_targets,ML_targets_prior,num,num_prior=experiment4(N,lamb,d,targets,noise_std[i],ml_resol[j],prior_means,prior_sigma,flag_plot)
        ML_MSE[i,j,0],_=evaluate(true_targets,ML_targets)
        ML_MSE[i,j,1],_=evaluate(true_targets,ML_targets_prior)
        print(f'noise_std:{noise_std[i]},ML1 resolution:{ml_resol[j]/lamb}wavelength')
        print(f'ML_mean_Squared_loss: {ML_MSE[i,j,0]:.4}')
        print(f'ML_mean_Squared_loss_prior: {ML_MSE[i,j,1]:.4}')
        print(f'number of grids: {num},{num_prior}')

In [ ]:
plt.figure()
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,0,0],ML_MSE[1,0,0],ML_MSE[2,0,0]]),'+-r')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,1,0],ML_MSE[1,1,0],ML_MSE[2,1,0]]),'+-b')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,2,0],ML_MSE[1,2,0],ML_MSE[2,2,0]]),'+-g')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,0,1],ML_MSE[1,0,1],ML_MSE[2,0,0]]),'+--r')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,1,1],ML_MSE[1,1,1],ML_MSE[2,1,1]]),'+--b')
plt.plot(np.array([60,40,20]),np.array([ML_MSE[0,2,1],ML_MSE[1,2,1],ML_MSE[2,2,1]]),'+--g')

plt.legend(['ML grid_size=0.5wavelength','ML grid_size=0.25wavelength','ML grid_size=0.125wavelength','ML grid_size=0.5wavelength+prior','ML grid_size=0.25wavelength+prior','ML grid_size=0.125wavelength+prior'])
plt.ylim([-70,-30])

plt.xlabel('σ-2 [dB]')
plt.ylabel('MSE [dB]')
plt.grid()
# plt.savefig('C:/Users/73160/OneDrive/项目/project/Weekly reports/Sp_6/Exp4_MSE_Result.png')
# np.savetxt('Exp4_result_ML_MSE.txt',ML_MSE[:,:,0],fmt='%.03f')
# np.savetxt('Exp4_result_ML_MSE_prior.txt',ML_MSE[:,:,1],fmt='%.03f')